<a href="https://colab.research.google.com/github/CLmarco01/Tesina-Change-Detection/blob/main/Tesina_Change_Detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Autore**: Marco Giangreco, 1938091

**Descrizione del progetto**: Risolvere il problema della binary change detection, tecnica statistica utilizzata per identificare cambiamenti o anomalie in una sequenza di dati.

##Import Libraries

In [1]:
import pandas as pd
import numpy as np

##Load the dataset

Dataset di riferimento (OSCD - ONERA SATELLITE CHANGE DETECTION): https://rcdaudt.github.io/oscd/